Создадим простой пример работы с бд

In [ ]:
from langchain_neo4j import Neo4jGraph

from langchain_community.vectorstores import Neo4jVector
from langchain_community.graphs import Neo4jGraph
from langchain_community.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_gigachat import GigaChatEmbeddings, GigaChat
from langchain_community.vectorstores import Chroma


from langchain_community.document_loaders import PyPDFLoader, TextLoader, Docx2txtLoader

def load_documents(file_paths, max_pages=None):
    """Загружает документы разных форматов с ограничением по страницам."""
    documents = []
    
    for file_path in file_paths:
        if file_path.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
            # Загружаем все страницы, но затем ограничиваем
            loaded_docs = loader.load()
            
            # Ограничиваем количество страниц, если указан max_pages
            if max_pages is not None and max_pages > 0:
                loaded_docs = loaded_docs[11:max_pages]
                
            print(f"📄 Загружено {len(loaded_docs)} страниц из {file_path}")
            documents.extend(loaded_docs)
            
        elif file_path.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
            # Для DOCX ограничиваем другим способом
            docs = loader.load()
            documents.extend(docs)
            print(f"📄 Загружен DOCX документ: {file_path}")
            
        elif file_path.endswith('.txt'):
            loader = TextLoader(file_path, encoding='utf-8')
            docs = loader.load()
            documents.extend(docs)
            print(f"📄 Загружен TXT документ: {file_path}")
            
        else:
            print(f"❌ Формат {file_path} не поддерживается")
            continue
    
    total_pages = len([doc for doc in documents if hasattr(doc, 'metadata') and 'page' in doc.metadata])
    print(f"📊 Итого загружено: {len(documents)} документов/чанков, из них примерно {total_pages} страниц")
    return documents


def split_documents(documents, chunk_size=500, chunk_overlap=150):
    """Разделяет документы на перекрывающиеся фрагменты"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", " ", ""]
    )
    
    return text_splitter.split_documents(documents)


def create_neo4j_vector_store(documents, embeddings, graph, index_name="textbook_chunks"):
    """
    Создает векторное хранилище в Neo4j, используя переданное подключение.
    """
    chunks = split_documents(documents)
    

    vector_store = Neo4jVector.from_documents(
        documents=chunks,
        embedding=embeddings,
        graph=graph,
        index_name=index_name,
        node_label="TextbookChunk",
        text_node_property="text",
        embedding_node_property="embedding"
    )
    
    print(f"✅ Создано {len(chunks)} чанков в Neo4j с векторным индексом '{index_name}'")
    return vector_store


def get_existing_vector_store(graph, embeddings):
    """Подключается к существующему векторному хранилищу в Neo4j."""
    
    # Параметры подключения (ДОЛЖНЫ СОВПАДАТЬ с теми, что использовались при создании)

    
    index_name = "informatics_textbook"
    node_label = "TextbookChunk"
    text_property = "text"
    embedding_property = "embedding"
    
    print("📂 Подключаюсь к существующему векторному хранилищу...")
    
    vector_store = Neo4jVector.from_existing_index(
        embedding=embeddings,
        graph=graph,
        index_name=index_name,
        node_label=node_label,
        text_node_property=text_property,
        embedding_node_property=embedding_property
    )
    return vector_store
    
if __name__ == "__main__":

    url = "neo4j://127.0.0.1:7687"
    username = "neo4j"
    password = "qwertyas"
    database_name = "neo4j"

    graph = Neo4jGraph(
        url=url,
        username=username,
        password=password,
        database=database_name,
        driver_config={"database": database_name},
        refresh_schema=False 
    )
        
    # Инициализация эмбеддинговой модели (GigaChat)
    embeddings = GigaChatEmbeddings(
        credentials="MD==",
        verify_ssl_certs=False
    )
    
    # Инициализация LLM (GigaChat) для извлечения знаний
    llm = GigaChat(
        credentials="MDE5OWF", 
        model="GigaChat",
        verify_ssl_certs=False
    )
    
    # 2. Загрузка документов (ваш существующий код)
    # documents = load_documents(["book2.pdf"],)
    
    # # Создаем бд если ее не было
    # vector_store = create_neo4j_vector_store(
    #     documents=documents,
    #     embeddings=embeddings,
    #     graph=graph,
    #     index_name="informatics_textbook"
    # )
    vector_store = get_existing_vector_store(graph, embeddings)
    


    question = "Как решить Слау матричным способом ?"
    similar_chunks = vector_store.similarity_search(question, k=3)
    
    for i, chunk in enumerate(similar_chunks, 1):
        print(f"--- Результат {i} ---")
        print(chunk.page_content)
        print(f"Источник: {chunk.metadata.get('source', 'неизвестно')}\n")


C:\Users\User\AppData\Local\Temp\ipykernel_10036\891801241.py:114: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  graph = Neo4jGraph(


📂 Подключаюсь к существующему векторному хранилищу...
--- Результат 1 ---
ных членов: 
15) [15—20 }_[-5 
5] |-15+425] [10] ° 
_5 
4. Присвоим полученный результат вектору-столбцу Х: Х = :0 | 
Для решения СЛАУ матричным методом с помощью МАТЕАВ в команд- 
ное окно следует ввести следующую команду: 
>> Х = 11% ([5,4;1,1])*[15;5] 
Х = 
1—4 А'.В= |. 5 
-5 
10 
Главным недостатком матричного метода решения СЛАУ является не- 
обходимость определения обратной матрицы, для вычисления которой тре- 
буется значительное время. Из-за этого матричный метод решения СЛАУ
Источник: book2.pdf

--- Результат 2 ---
ентов системы линейных алгебраических уравнений, то данная система не- 
совместная, т.е. не имеет решений. 
4.2.3. Матричный метод решения СЛАУ 
Используя свойства матриц, приведенную выше матричную форму 
СЛАУ можно привести к следующему виду: 
А" (АХ) = А'В <> (А'А)Х = АПВ <> Х=АТВ, 
где А' — обратная квадратная матрица. 
Следовательно, для решения СЛАУ матричным методом необходимо 
выполнит

Сверху была простоа реализациям   и заполнение бд, теперь преступим к более сложному , посоммтрим как это все было бы, если у нас была доабваная llm модель т.е осознание запроса


In [2]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_gigachat import GigaChat
from langchain_community.vectorstores import Neo4jVector
from langchain_gigachat.embeddings import GigaChatEmbeddings
    

# 1. Определяем промпт для учебника информатики
PROMPT_TEMPLATE = """Ты — помощник по учебнику информатики. Используй ТОЛЬКО предоставленный контекст для ответа.
Если в контексте нет информации для ответа, скажи: "В предоставленных материалах этого нет."

Контекст:
{context}

Вопрос: {question}
Ответ:"""

PROMPT = PromptTemplate(
    template=PROMPT_TEMPLATE,
    input_variables=["context", "question"]
)


# 2. Создаем цепочку RAG
def create_rag_chain(vector_store, llm):
    """Создает RAG-цепочку с использованием существующего векторного хранилища."""
    
    retriever = vector_store.as_retriever(
        search_type="mmr",  # Maximal Marginal Relevance для разнообразия результатов
        search_kwargs={
            "k": 5,          # Сколько чанков возвращать
            "fetch_k": 10,   # Сколько чанков рассматривать первоначально
            "lambda_mult": 0.7  # Баланс между релевантностью (1.0) и разнообразием (0.0)
        }
    )
    
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",  # Просто "набиваем" все чанки в контекст
        retriever=retriever,
        chain_type_kwargs={"prompt": PROMPT},
        return_source_documents=True
    )
    
    return qa_chain

# 3. Функция для вопросов
def ask_question(qa_chain, question):
    """Задает вопрос RAG-системе."""
    result = qa_chain.invoke({"query": question})
    
    # Извлекаем ответ и источники
    answer = result["result"]
    source_docs = result["source_documents"]
    
    # Уникальные источники
    sources = list(set([doc.metadata.get("source", "Unknown") for doc in source_docs]))
    
    print(f"🤔 Вопрос: {question}")
    print(f"💡 Ответ: {answer}")
    print(f"📚 Источники: {', '.join(sources)}")
    print(f"📄 Использовано фрагментов: {len(source_docs)}\n")
    
    return {
        "answer": answer,
        "sources": sources,
        "relevant_chunks": len(source_docs)
    }

# 4. ПОЛНЫЙ ПРИМЕР ИСПОЛЬЗОВАНИЯ
if __name__ == "__main__":
    # A. Подключаемся к готовой векторной базе

    print("🔗 Подключаюсь к векторному хранилищу Neo4j...")
    vector_store = get_existing_vector_store(graph, embeddings)


    print("🔄 Создаю RAG-систему...")
    qa_chain = create_rag_chain(vector_store, llm)
    
    # D. Задаем вопросы!
    print("\n" + "="*50)
    
    # Тестовые вопросы по учебнику информатики
    questions = [
        "Как решить Слау матричным способом ?"
    ]
    
    for question in questions:
        ask_question(qa_chain, question)

🔗 Подключаюсь к векторному хранилищу Neo4j...
📂 Подключаюсь к существующему векторному хранилищу...
🔄 Создаю RAG-систему...

🤔 Вопрос: Как решить Слау матричным способом ?
💡 Ответ: **Последовательность шагов для решения СЛАУ матричным методом:**

1. **Проверка невырожденности матрицы:**  
   Необходимо убедиться, что исходная матрица $A$ не является вырожденной (определитель матрицы не равен нулю).
   
2. **Приведение матричной формы СЛАУ:**  
   Исходную систему уравнений представляем в матричном виде:  
   $$ A \cdot X = B $$

3. **Находим обратную матрицу $A'$:**  
   Обратная матрица $A'$ должна существовать и использоваться для нахождения вектора решений $X$.

4. **Вычисление вектора решений:**  
   Решением системы будет вектор $X$, который находится по формуле:  
   $$ X = A'B $$

5. **Использование системы MATLAB (МАТГАЛ):**  
   Для реализации матричного метода в системе MATLAB вводятся команды вида:  
   ```matlab
   >> X = inv(A)*B
   ```
   где `inv(A)` — операция обращения

Отлично, теперь нам надо сделать связи, что мы будем делать 
1) Определить типы связи , т.к мы не буржуи за нас это сделает дргая модель , а мы просто скажем ей спасибо и создадим
2) Также опрпеделим сами типы нодов, и неачнем постепенно реализовывать наше чудо 
Тогда создадим корректтный json для описания связей и типа 


In [ ]:
# 1. ПРОМПТ для извлечения LearningTask (Учебные задачи)
LEARNING_TASK_PROMPT = """Из текста найди УЧЕБНУЮ ЗАДАЧУ или практическое задание.
Верни JSON: {"goal": "формулировка задачи", "difficulty": "easy/medium/hard"}"""

# 2. ПРОМПТ для извлечения OctaveElement (Описания элементов Octave)
OCTAVE_ELEMENT_PROMPT = """Найди ОПИСАНИЕ или ПОЯСНЕНИЕ элемента Octave в тексте.
Это может быть объяснение функции, оператора, синтаксиса.
Верни JSON: {"name": "имя элемента", "type": "function/operator/syntax", "description": "пояснение"}"""

# 3. ПРОМПТ для связи RELATED_TO между OctaveElement
RELATED_TO_PROMPT = """Найди в тексте СВЯЗАННЫЕ элементы Octave (например, одна функция используется в другой).
Верни JSON: {"source": "первый элемент", "target": "второй элемент", "reason": "почти связаны"}"""



In [7]:
import re
import json

def parse_gigachat_json(response):
    """
    Извлекает JSON из ответа GigaChat.
    Просто ищет {...} в тексте.
    """
    
    # Получаем текст
    text = response.content if hasattr(response, 'content') else str(response)
    
    # Ищем { ... }
    match = re.search(r'\{.*\}', text, re.DOTALL)
    
    if match:
        try:
            return json.loads(match.group(0))
        except:
            # Если не парсится, возвращаем ошибку
            return {"error": "Невалидный JSON"}
    
    return {"error": "JSON не найден"}

# Использование:
# response = llm.invoke("промпт")
# data = parse_gigachat_json(response)

In [10]:
import json 
import pandas as pd
import os

# Обновлённые промпты с четкими условиями
LEARNING_TASK_PROMPT = """Верни ТОЛЬКО JSON без пояснений. Проанализируй текст и найди УЧЕБНУЮ ЗАДАЧУ или практическое задание для студента.
Если в тексте явно есть задача (например: "создайте...", "решите...", "постройте...", "найдите..."), 
верни JSON: {"goal": "формулировка задачи", "difficulty": "easy/medium/hard"}.

Если в тексте НЕТ задачи, верни JSON: {"goal": null}."""

OCTAVE_ELEMENT_PROMPT = """Верни ТОЛЬКО JSON без пояснений. Проанализируй текст и найди ОПИСАНИЕ элемента Octave.
Элемент Octave - это: 
1. Функция (например: plot(), zeros(), det())
2. Оператор (например: +, *, \, .*)
3. Синтаксис (например: A = [1 2; 3 4])
4. Команда или функция Octave/Matlab

Если в тексте описывается элемент Octave, верни JSON: {"name": "имя элемента", "type": "function/operator/syntax", "description": "пояснение"}.

Если в тексте НЕТ описания элемента Octave, верни JSON: {"name": null}."""

# Улучшенный код с проверкой на null
def process_chunks_simple(vector_store, llm, n=10, max_chunks=1):
    """Простая обработка: 2 типа узлов, 1 тип связи"""
    
    all_chunks = vector_store.similarity_search("", k=2715)
    
    selected_indices = range(0, len(all_chunks), n)
    selected_chunks = [all_chunks[i] for i in selected_indices[:max_chunks]]
    
    print(f"📊 Всего чанков: {len(all_chunks)}")
    print(f"🎯 Обрабатываю каждый {n}-й: {len(selected_chunks)} чанков")

    tasks = []       # LearningTask
    elements = []    # OctaveElement  
    relations = []   # ТОЛЬКО FOLLOWS
    
    # Простой список всех узлов в порядке их извлечения
    all_nodes_in_order = []
    
    for i, chunk in enumerate(selected_chunks):
        chunk_text = chunk.page_content[:300]
        chunk_id = i * n
        
        try:
            # 1. Извлекаем задачу (если есть)
            task_response = llm.invoke(f"{LEARNING_TASK_PROMPT}\nТекст: {chunk_text}")
            task_json = parse_gigachat_json(task_response)
            
            # Проверяем, что goal не null и не пустая строка
            if task_json.get("goal") and task_json["goal"] is not None and task_json["goal"].strip():
                task_id = f"lt_{len(tasks):04d}"
                task = {
                    "id": task_id,
                    "label": "LearningTask",
                    "goal": task_json["goal"],
                    "difficulty": task_json.get("difficulty", "medium"),
                    "chunk_order": chunk_id
                }
                tasks.append(task)
                all_nodes_in_order.append(task)  # Добавляем в общий список
                print(f"   ✅ Задача: {task_json['goal'][:50]}...")
            
            # 2. Извлекаем элемент Octave (если есть)
            octave_response = llm.invoke(f"{OCTAVE_ELEMENT_PROMPT}\nТекст: {chunk_text}")
            octave_json = json.loads(octave_response.content)
            
            # Проверяем, что name не null и не пустая строка
            if octave_json.get("name") and octave_json["name"] is not None and octave_json["name"].strip():
                element_id = f"oe_{len(elements):04d}"
                element = {
                    "id": element_id,
                    "label": "OctaveElement",
                    "name": octave_json["name"],
                    "type": octave_json.get("type", "unknown"),
                    "description": octave_json.get("description", ""),
                    "chunk_order": chunk_id
                }
                elements.append(element)
                all_nodes_in_order.append(element)  # Добавляем в общий список
                print(f"   ✅ Элемент: {octave_json['name']}")
            
        except Exception as e:
            print(task_response)
            print(f"   ❌ Ошибка: {str(e)[:50]}")
            continue
    
    print(f"\n✅ Итого извлечено: {len(tasks)} задач, {len(elements)} элементов")
    print(f"📋 Всего узлов в порядке: {len(all_nodes_in_order)}")
    
    # 3. Просто создаем связи FOLLOWS между соседними узлами в общем списке
    for i in range(len(all_nodes_in_order) - 1):
        current = all_nodes_in_order[i]
        next_node = all_nodes_in_order[i + 1]
        
        # Проверяем, что это разные узлы
        if current["id"] == next_node["id"]:
            continue
        
        relations.append({
            "start_id": current["id"],
            "end_id": next_node["id"],
            "type": "FOLLOWS",
            "reason": f"Следует за {current.get('goal', current.get('name', ''))[:30]}...",
            "weight": 1.0
        })
    
    print(f"🔗 Создано связей FOLLOWS: {len(relations)}")
    
    # 4. Сохраняем в CSV
    save_to_csv_simple(tasks, elements, relations, n)
    
    return len(tasks), len(elements), len(relations)

def create_readable_names(tasks, elements):
    """Создает понятные имена для узлов"""
    for task in tasks:
        task["name"] = f"Задача: {task['goal'][:50]}{'...' if len(task['goal']) > 50 else ''}"
    
    for element in elements:
        element["name"] = element["name"]  # уже есть имя
    
    return tasks, elements

def save_to_csv_simple(tasks, elements, relations, n):
    """Сохраняет только 2 типа узлов и 1 тип связи"""
    folder = f"simple_data_1in{n}"
    os.makedirs(folder, exist_ok=True)
    
    # Добавляем читаемые имена
    tasks, elements = create_readable_names(tasks, elements)
    
    # 1. LearningTask
    if tasks:
        pd.DataFrame(tasks).to_csv(f"{folder}/learning_tasks.csv", index=False, encoding='utf-8')
    
    # 2. OctaveElement
    if elements:
        pd.DataFrame(elements).to_csv(f"{folder}/octave_elements.csv", index=False, encoding='utf-8')
    
    # 3. FOLLOWS связи
    if relations:
        pd.DataFrame(relations).to_csv(f"{folder}/follows_relations.csv", index=False, encoding='utf-8')
    
    print(f"\n💾 Сохранено в '{folder}/':")
    print(f"   📄 learning_tasks.csv: {len(tasks)} задач")
    print(f"   📄 octave_elements.csv: {len(elements)} элементов")
    print(f"   📄 follows_relations.csv: {len(relations)} связей FOLLOWS")
    
    # 4. Дополнительно: создаем файлы для импорта в Neo4j
    create_neo4j_import_files(tasks, elements, relations, folder)

def create_neo4j_import_files(tasks, elements, relations, folder):
    """Создает файлы в формате для Neo4j LOAD CSV"""
    
    # 1. Объединенный файл узлов
    nodes_data = []
    
    for task in tasks:
        nodes_data.append({
            "nodeId:ID": task["id"],
            "label:LABEL": "LearningTask",
            "name": task.get("name", task["id"]),
            "goal": task["goal"],
            "difficulty": task["difficulty"],
            "chunk_order:int": task["chunk_order"]
        })
    
    for element in elements:
        nodes_data.append({
            "nodeId:ID": element["id"],
            "label:LABEL": "OctaveElement",
            "name": element["name"],
            "type": element["type"],
            "description": element.get("description", ""),
            "chunk_order:int": element["chunk_order"]
        })
    
    if nodes_data:
        pd.DataFrame(nodes_data).to_csv(f"{folder}/neo4j_nodes.csv", index=False, encoding='utf-8')
    
    # 2. Файл связей
    if relations:
        relations_data = []
        for rel in relations:
            relations_data.append({
                ":START_ID": rel["start_id"],
                ":END_ID": rel["end_id"],
                ":TYPE": rel["type"],
                "reason": rel.get("reason", ""),
                "weight:float": rel.get("weight", 1.0)
            })
        
        pd.DataFrame(relations_data).to_csv(f"{folder}/neo4j_relations.csv", index=False, encoding='utf-8')
    
    print(f"   📄 neo4j_nodes.csv: {len(nodes_data)} узлов (для импорта в Neo4j)")
    print(f"   📄 neo4j_relations.csv: {len(relations)} связей (для импорта в Neo4j)")

<>:15: SyntaxWarning: invalid escape sequence '\,'
<>:15: SyntaxWarning: invalid escape sequence '\,'
C:\Users\User\AppData\Local\Temp\ipykernel_10036\1788457549.py:15: SyntaxWarning: invalid escape sequence '\,'
  2. Оператор (например: +, *, \, .*)


In [ ]:
process_chunks_simple(vector_store, llm, n=10, max_chunks=3)


📊 Всего чанков: 2715
🎯 Обрабатываю каждый 10-й: 10 чанков
   ✅ Задача: нет учебной задачи...
content='{"goal": "нет учебной задачи", "difficulty": null}' additional_kwargs={} response_metadata={'token_usage': {'prompt_tokens': 202, 'completion_tokens': 15, 'total_tokens': 217, 'precached_prompt_tokens': 110}, 'model_name': 'GigaChat:2.0.28.2', 'x_headers': {'x-request-id': 'bce5716f-bcea-4ad2-85a0-0c764ff5e27e', 'x-session-id': '92292d7f-dad4-4758-bfc6-80c0f77d8d1b', 'x-client-id': None}, 'finish_reason': 'stop'} id='bce5716f-bcea-4ad2-85a0-0c764ff5e27e' usage_metadata={'output_tokens': 15, 'input_tokens': 202, 'total_tokens': 217, 'input_token_details': {'cache_read': 110}}
   ❌ Ошибка: Expecting value: line 1 column 1 (char 0)
   ✅ Задача: нет учебной задачи...
   ✅ Задача: нет учебной задачи...


C:\Users\User\AppData\Local\Temp\ipykernel_10036\1788457549.py:15: SyntaxWarning: invalid escape sequence '\,'
  2. Оператор (например: +, *, \, .*)


KeyboardInterrupt: 